# Map Wrapper Development for Scipy Differential Equation

Proof of concept notebook for first Scipy, since it only requires map not the o

Error_function is a pass function.
Our "map" takes care of initiating the whole neuron group, calculating the errors and returing the value.

```
params, fits, error = fit_traces(model = model, input_var = 'v', output_var = 'I',\
                                 input = input_traces, output = output_traces,
                                 dt = 0.1*ms, g = [1*nS, 30*nS], E = [-20*mV,100*mV],
                                 tol = 1e-6)
```

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver


In [2]:
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod



#### Artificial Data

In [3]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])


In [4]:
params2 = np.array([[ 1.12735759e-08,  8.81556360e-02],
 [ 9.04858783e-09,  8.03343489e-02],
 [ 1.60635907e-08,  1.59710651e-03],
 [ 1.66254878e-09,  2.95479385e-02],
 [ 1.80869973e-08,  2.50218013e-02],])

### Input / Output

In [5]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

# Create target current traces
output_traces = 10*nS*input_traces

input = input_traces
output = output_traces

### Parameters 

In [6]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

### Calculation for inner parameters

In [7]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [8]:
N

3

In [9]:
Ntraces

1

### Setup The Model for Optimization

In [10]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [11]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.01s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [12]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [13]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [14]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

# Record error
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

store()


In [15]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Dev Map Optimization 

### Create the Dictonaries of Parameters

In [16]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [17]:
d = parameters_dict(params)
d

{'g': array([1.80869973e-08, 1.88373085e-08, 1.88373085e-08]),
 'E': array([0.0250218 , 0.09895599, 0.09895599])}

In [18]:
neurons.get_states()

{'N': array(3),
 'i': array([0, 1, 2], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([0., 0., 0.]) * siemens,
 'E': array([0., 0., 0.]) * volt,
 'total_error': array([0., 0., 0.]) * amp2}

In [19]:
restore()
neurons.set_states(d, units=False)
neurons.get_states()

{'N': array(3),
 'i': array([0, 1, 2], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([18.0869973, 18.8373085, 18.8373085]) * nsiemens,
 'E': array([25.0218013, 98.9559934, 98.9559934]) * mvolt,
 'total_error': array([0., 0., 0.]) * amp2}

In [20]:
run(duration, namespace = {})

In [21]:
e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
e = mean(e.reshape((N,Ntraces)),axis=1)
array(e)

array([2.04818928e-19, 3.47473674e-18, 3.47473674e-18])

## All of the above into one hard-coded function

In [22]:
start_scope()

In [23]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [24]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [25]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [26]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [37]:
def calc_error(func, params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [58]:
def error_pass(x):
    print('x', x)
    pass

In [59]:
calc_error(error_pass, params2)

[[1.12735759e-08 8.81556360e-02]
 [9.04858783e-09 8.03343489e-02]
 [1.60635907e-08 1.59710651e-03]
 [1.66254878e-09 2.95479385e-02]
 [1.80869973e-08 2.50218013e-02]]


array([9.87696585e-19, 5.28401654e-19, 6.58192637e-22, 2.41325473e-21,
       2.04818928e-19])

## Try The Same with Differential Evolution from Scipy

In [60]:
start_scope()

In [61]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [62]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [63]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [64]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [65]:
bounds = [(1, 30), (-20, 100)]

In [67]:
differential_evolution(error_pass, bounds, updating='deferred', workers=calc_error, maxiter=1, polish=False)

[[ 22.16439521  -4.21995571]
 [ 17.47509532  47.89894078]
 [  7.4431031   54.47883476]
 [ 25.05109863  64.35183627]
 [  2.95555495 -11.73461072]
 [ 11.08506172  70.59750338]
 [  1.18846565 -19.42179613]
 [ 19.49467543  79.30350439]
 [ 19.15130525  84.37599135]
 [ 14.68836896  51.12754924]
 [ 13.32388274 -13.20221226]
 [ 28.18241024  15.95163971]
 [ 12.40904962  43.97926051]
 [ 26.58186904  26.55231166]
 [  9.52087785   5.00241707]
 [ 20.47870047  11.61107073]
 [  5.52320195  62.84003161]
 [ 17.26921906  33.30606912]
 [ 25.68405443  92.31175496]
 [  2.69084168   1.72301698]
 [  6.11153358  28.71397042]
 [ 27.60131301  21.38256668]
 [ 16.39326534  91.09273534]
 [ 23.8651959   75.95543893]
 [ 29.97140747  17.42393459]
 [ 10.22433925  82.40247847]
 [ 22.80691603  57.15152749]
 [  4.03738462  97.06837292]
 [ 14.16470718  39.72960951]
 [  8.3472486   -3.26794749]]
[[11.60909834 -8.52662244]
 [ 1.70718266 -4.98756762]
 [27.70406359 90.78722883]
 [ 5.03434317 28.98282161]
 [ 9.15745003 -6.9073

     fun: 4.726608611011728
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 60
     nit: 1
 success: False
       x: array([ 4.03738462, -0.53848635])